# Scope of work

1) Import all necessary libraries and modules.  
2) First we need to obtain inforamtion about all available cars from the first main pages.  
3) Create a SQL database and export there gathered info about cars from the main pages.  
4) Then we will call module dealers_cars to acquire links to every dealer's list of cars from all main pages.  
5) Now we will repeat the same procedure as in the 1st step but to the every dealer's cars list.  
6) Add extracted data to a created SQL database

## 1. Imort of libraries and modules

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from sqlalchemy import create_engine
import json
import dealers_cars
import marks

## 2. Acquring cars' information from the first main pages

In [2]:
# Here we provide URL to the very first main page.

In [3]:
url = 'https://www.autoscout24.com/lst?atype=C&desc=0&sort=standard&source=homepage_search-mask&ustate=N%2CU'
html = requests.get(url).text
soup = BeautifulSoup(html, 'lxml')

In [4]:
#get the number of total pages on the web site
def total_pages(soup):
    divs = soup.find('div', attrs={'class':'ListPage_pagination__v_4ci'})
    pages = divs.find_all('button', attrs={'class':'FilteredListPagination_button__41hHM'})[-2].text
    total_pages = int(pages)
    return total_pages

In [5]:
#get a dictionary of all URLs from every main page
def pages_urls(url):
    all_pages = {}
    for i in range(1, total_pages(soup) + 1):
        url_parts = url.split('&')
        url_parts[2] = f'page={i}'
        url = '&'.join(url_parts)
        all_pages[i]=url
    return all_pages

all_pages=pages_urls(url)

In [6]:
#here we create a list of html codes as soup elements about all pages
def html_list():
    soups_list = []
    for k in all_pages:
        soups_list.append(BeautifulSoup(requests.get(all_pages[k]).text, 'lxml'))
    return soups_list

soups_list = html_list()

In [207]:
# here we create blank lists to populate it later with cars' info
cars = []
characteristics = []
prices = []
locations = []

#this function scraps over the website in order to extract specific information about each car (characteristics, prices etc)
def parcing (tag, attr, df):
    for element in soups_list:
        info = element.find_all(tag, attrs={'class':attr})
        for i in info:
            df.append(i.get_text())
    return df

In [208]:
#this function populate previously created blank lists with cars' info
def info():
    c = parcing('a', 'ListItem_title__znV2I ListItem_title_new_design__lYiAv Link_link__pjU1l', cars)
    ch = parcing('div', 'VehicleDetailTable_container__mUUbY', characteristics)
    p = parcing('p', 'Price_price__WZayw PriceAndSeals_current_price__XscDn', prices)
    l = parcing('span', 'SellerInfo_address__txoNV', locations)
    return c, ch, p, l

In [209]:
cars, characteristics, prices, locations = info()

In [210]:
def formating():
    #here we extract only car's mark and maodel
    for i in range(len(cars)):
        cars[i] = cars[i].split('\xa0')[0]
    fuel_types = ['Gasoline','Diesel','Ethanol','Electric','Hydrogen','LPG','CNG','Electric/Gasoline','Others',
              'Electric/Diesel']
    fuel_pattern = '|'.join(fuel_types)
    gear = ['Automatic','Manual','Semi-automatic']
    gear_pattern = '|'.join(gear)
    #here we extract specific patterns of each car characteristics. The initial text that was extracted from web scraping
    #contains too much unrelated data
    for i in range(len(characteristics)):
        patterns = [r'\d{1,3}(?:,\d{3})*\s?km', f'({gear_pattern})', r'\d{1,2}/\d{4}', f'({fuel_pattern})', r'\d{1,4}\s?hp']
        characteristics[i] = [re.search(pattern, characteristics[i]).group(0).replace(',', '').replace(' km', '')
                              .replace(' hp', '').strip() if re.search(pattern, characteristics[i])
                              else None for pattern in patterns]
    #here we extract integer from price text
    for i in range(len(prices)):
        prices[i] = int(re.sub(r'\D', '', prices[i]))
        
    #here we extract only country abbreveation
    for i in range(len(locations)):
        try:
            locations[i] = locations[i].split('• ')[1].split('-')[0]
        except:
            locations[i] = locations[i].split('-')[0]
            
    return cars, characteristics, prices, locations

cars, characteristics, prices, locations = formating()

In [202]:
#for i in range(len(cars)):
#    cars[i] = cars[i].split(' ', 1)

In [224]:
cars

['Mercedes-Benz E 63 AMG',
 'Mercedes-Benz AMG GT',
 'Audi RS6',
 'Mercedes-Benz E 63 AMG',
 'BMW 520',
 'Porsche 911',
 'Audi TT',
 'Audi Q5',
 'Ferrari 296',
 'Porsche Panamera',
 'Audi A3',
 'BMW M5',
 'Ferrari',
 'Volkswagen Caddy',
 'Ford Mustang',
 'Opel Antara',
 'Porsche Cayman',
 'BMW M850',
 'Mercedes-Benz C 220',
 'Maybach',
 'Audi RS6',
 'Skoda Kodiaq',
 'Audi Q3',
 'Maserati MC20',
 'Ferrari 812',
 'Mercedes-Benz S 580',
 'Mercedes-Benz S 580',
 'Mercedes-Benz AMG GT',
 'Mercedes-Benz E 55 AMG',
 'Lamborghini Aventador',
 'Skoda Kodiaq',
 'Audi RS6',
 'Audi A6',
 'SEAT Leon',
 'Audi Q3',
 'BMW M5',
 'Opel Astra',
 'Volkswagen Golf GTE',
 'BMW M3',
 'Mercedes-Benz CLA 200',
 'Others JAC',
 'Opel Corsa',
 'Bentley Continental GT',
 'Cupra Formentor',
 'Maserati Ghibli',
 'BMW 520',
 'BMW M4',
 'Chrysler PT Cruiser',
 'Audi A4',
 'BMW M5',
 'Audi Q2',
 'Cupra Born',
 'BMW 118',
 'Audi RS6',
 'Mercedes-Benz C 200',
 'Lamborghini Huracan',
 'Volkswagen T6.1 Multivan',
 'Porsche

In [223]:
for car in cars:
    words = car.split()
    # Check if there are more than two words (make and model)
    if len(words) > 2:
        make = '-'.join(words[:2]) # Replace spaces with hyphens in the make (first two words)
        modified_car_name = make + ' ' + ' '.join(words[2:]) # Join the modified make with the model part (the remaining words)
    else:
        # If there are only two or fewer words, replace spaces with hyphens in the entire name
        modified_car_name = ' '.join(words).replace(' ', '-')
        
    print(modified_car_name)    
        

Mercedes-Benz-E 63 AMG
Mercedes-Benz-AMG GT
Audi-RS6
Mercedes-Benz-E 63 AMG
BMW-520
Porsche-911
Audi-TT
Audi-Q5
Ferrari-296
Porsche-Panamera
Audi-A3
BMW-M5
Ferrari
Volkswagen-Caddy
Ford-Mustang
Opel-Antara
Porsche-Cayman
BMW-M850
Mercedes-Benz-C 220
Maybach
Audi-RS6
Skoda-Kodiaq
Audi-Q3
Maserati-MC20
Ferrari-812
Mercedes-Benz-S 580
Mercedes-Benz-S 580
Mercedes-Benz-AMG GT
Mercedes-Benz-E 55 AMG
Lamborghini-Aventador
Skoda-Kodiaq
Audi-RS6
Audi-A6
SEAT-Leon
Audi-Q3
BMW-M5
Opel-Astra
Volkswagen-Golf GTE
BMW-M3
Mercedes-Benz-CLA 200
Others-JAC
Opel-Corsa
Bentley-Continental GT
Cupra-Formentor
Maserati-Ghibli
BMW-520
BMW-M4
Chrysler-PT Cruiser
Audi-A4
BMW-M5
Audi-Q2
Cupra-Born
BMW-118
Audi-RS6
Mercedes-Benz-C 200
Lamborghini-Huracan
Volkswagen-T6.1 Multivan
Porsche-911
Audi-Q4 e-tron
Dodge-Charger
Audi-A8
Tesla-Model 3
Ferrari-SF90 Stradale
Porsche-Cayman
EVO-EVO3
Cupra-Born
Audi-Q2
Maybach
Opel-Vivaro
BMW-M2
Audi-Q3
BMW-M4
SsangYong-REXTON
Mercedes-Benz-AMG GT
Mercedes-Benz-G 63 AMG
Cupra-

In [204]:
space_names = []

In [205]:
for mark in marks_menu:
    if ' ' in mark:
        space_names.append(mark)

In [206]:
space_names

['TOP MAKES',
 'OTHER MAKES',
 'Alba Mobility',
 'Alfa Romeo',
 'Angelelli Automobili',
 'Ariel Motor',
 'Aston Martin',
 'De la Chapelle',
 'De Tomaso',
 'Devinci Cars',
 'DR Automobiles',
 'DS Automobiles',
 'Gac Gonow',
 'Giotti Victoria',
 'Great Wall',
 'Iso Rivolta',
 'Land Rover',
 'Lynk & Co',
 'Martin Motors',
 'MP Lafer',
 'MPM Motors',
 'Panther Westwinds',
 'Tazzari EV',
 'Town Life',
 'Vanden Plas']

In [196]:
for i in range(len(space_names)):
        space_names[i] = space_names[i].replace(' ','-')

In [197]:
space_names

['TOP-MAKES',
 'OTHER-MAKES',
 'Alba-Mobility',
 'Alfa-Romeo',
 'Angelelli-Automobili',
 'Ariel-Motor',
 'Aston-Martin',
 'De-la-Chapelle',
 'De-Tomaso',
 'Devinci-Cars',
 'DR-Automobiles',
 'DS-Automobiles',
 'Gac-Gonow',
 'Giotti-Victoria',
 'Great-Wall',
 'Iso-Rivolta',
 'Land-Rover',
 'Lynk-&-Co',
 'Martin-Motors',
 'MP-Lafer',
 'MPM-Motors',
 'Panther-Westwinds',
 'Tazzari-EV',
 'Town-Life',
 'Vanden-Plas']

In [195]:
for item2 in space_names:
    found = False
    for item1 in cars:
        if item2 in item1:
            print(item1)
            found = True
            break 

Alfa Romeo MiTo
Aston Martin Vanquish
Land Rover Range Rover


In [36]:
#marks_menu = marks.all_marks(url)

In [ ]:
'TOP MAKES', 'OTHER MAKES', 'Others'

In [22]:
marks_menu

['TOP MAKES',
 'Audi',
 'BMW',
 'Ford',
 'Mercedes-Benz',
 'Opel',
 'Volkswagen',
 'Renault',
 'OTHER MAKES',
 '9ff',
 'Abarth',
 'AC',
 'ACM',
 'Acura',
 'Aiways',
 'Aixam',
 'Alba Mobility',
 'Alfa Romeo',
 'Alpina',
 'Alpine',
 'Amphicar',
 'Angelelli Automobili',
 'Ariel Motor',
 'Artega',
 'Aspark',
 'Aspid',
 'Aston Martin',
 'Aurus',
 'Austin',
 'Austin-Healey',
 'Autobianchi',
 'BAIC',
 'Bedford',
 'Bellier',
 'Bentley',
 'Boldmen',
 'Bolloré',
 'Borgward',
 'Brilliance',
 'Bristol',
 'Bugatti',
 'Buick',
 'BYD',
 'Cadillac',
 'Caravans-Wohnm',
 'Carver',
 'Casalini',
 'Caterham',
 'Cenntro',
 'Changhe',
 'Chatenet',
 'Chery',
 'Chevrolet',
 'Chrysler',
 'Citroen',
 'CityEL',
 'Corvette',
 'Cupra',
 'Dacia',
 'Daewoo',
 'DAF',
 'Daihatsu',
 'Daimler',
 'Dallara',
 'Dangel',
 'De la Chapelle',
 'De Tomaso',
 'Delorean',
 'Devinci Cars',
 'DFSK',
 'Dodge',
 'Donkervoort',
 'DR Automobiles',
 'DS Automobiles',
 'Dutton',
 'e.GO',
 'Econelo',
 'Edran',
 'Elaris',
 'Embuggy',
 'EMC'

In [ ]:
# This function collects all previously formed lists and form 1 united dataframe in pandas
def to_pandas():
    #Here we transform our lists into pandas Series
    c = pd.Series(cars, name='Car')
    ch = pd.Series(characteristics)
    p = pd.Series(prices, name='Price [€]')
    l = pd.Series(locations, name='Location')
    # Create a DataFrame from the Series, which splits the lists into columns
    df = pd.DataFrame(ch.tolist(), columns=['Mileage [km]', 'Transmission', 'Registration [m/y]', 'Fuel', 'Power [hp]'])
    #Here we change data types of Mileage and Power to integers
    try:
        df['Mileage [km]'] = df['Mileage [km]'].astype('int')
        df['Power [hp]'] = df['Power [hp]'].astype('int')
    except:
        df['Power [hp]'] = df['Power [hp]'].fillna(0)
        df['Power [hp]'] = df['Power [hp]'].astype('int')
        df['Mileage [km]'] = df['Mileage [km]'].fillna(0)
        df['Mileage [km]'] = df['Mileage [km]'].astype('int')
    merged_df = pd.concat([c, df], axis=1)
    merged_df2 = pd.concat([merged_df,l], axis=1)
    main_pages_info = pd.concat([merged_df2,p], axis=1)
    return main_pages_info

In [ ]:
to_pandas()

## 3. Creating a SQL database and exporting parsed data there from the main pages

In [ ]:
# Load database configuration from a JSON file in order to avoid hardcoding sensible information
with open('postgres_configs.json') as config_file:
    config = json.load(config_file)

In [ ]:
#configurations to connect to a SQL database
db_user = config['postgres']['user']
db_password = config['postgres']['pwd']
db_host = config['postgres']['host']
db_port = config['postgres']['port']
db_name = config['postgres']['db']

In [ ]:
# Connection string
connection_string = '{}/{}@{}:{}/{}'.format(
    db_user,
    db_password,
    db_host,
    db_port,
    db_name
)

In [ ]:
#configurations to connect to a SQL database
#db_config = {
#    'user': 'postgres', 
#    'pwd': 'austria011020',
#    'host': 'localhost',
#    'port': 5432,
#    'db': 'cars_small'
#} 

In [ ]:
#connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
#    db_config['user'],
#    db_config['pwd'],
#    db_config['host'],
#    db_config['port'],
#    db_config['db'],
#)

In [ ]:
#engine = create_engine(connection_string)

In [ ]:
#with engine.connect() as conn:
#    conn.execute('CREATE SCHEMA IF NOT EXISTS cars_small;')

In [ ]:
#with engine.connect() as conn:
#    to_pandas().to_sql(name='cars_info', con=conn, if_exists='replace', index=False)

## 2. Acquiring links to every dealer's list of cars from all main pages.

In [ ]:
# We call the module dealers_cars and assign the URL from the first main page
dealers_cars_list = dealers_cars.sel_pars(url)

In [ ]:
dealers_cars_list

In [ ]:
s = set(dealers_cars_list)

In [ ]:
s

In [ ]:
dealers_cars_list[0]

In [ ]:
url0 = dealers_cars_list[0]
html0 = requests.get(url0).text
soup0 = BeautifulSoup(html0, 'lxml')

In [ ]:
total_pages(soup0)